In [2]:
import matplotlib
matplotlib.use('Agg')
from eam_core.yaml_runner import *
%matplotlib notebook

[root                ][INFO   ]  Configured logging from /home/jw17943/dimpact/eam-core-provenance/src/eam_core/logconf.yml (log_configuration.py:30)
[root                ][INFO   ]  Logging already configured. (log_configuration.py:15)


In [3]:
import matplotlib.pyplot as plt

In [5]:
args = {'yamlfile':'../../tests/models/valid.yml', 'comment':'jupyter','local':True,       'filetype':'pdf','docker':True }

class argsview(object):
    def __init__(self, d):
        self.__dict__ = d
        
args = argsview(args) 


yamlfile = args.yamlfile
yaml_struct = None
with open(yamlfile, 'r') as stream:
    try:
        yaml_struct = yaml.load(stream)
    except yaml.YAMLError as exc:
        logger.error(f'Error while loading yaml file {yamlfile} {exc}')
        sys.exit(1)

/tmp/ipykernel_383173/465861469.py:14: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  yaml_struct = yaml.load(stream)


In [7]:
%%capture
args = {'yamlfile':'../../tests/models/valid.yml', 'comment':'jupyter','local':True,       'filetype':'pdf','docker':True }

class argsview(object):
    def __init__(self, d):
        self.__dict__ = d
        
args = argsview(args)                                        


logger.info(f"Running with parameters {args}")
model_run_base_directory, simulation_run_description, yaml_struct = load_configuration(args)


scenarios = yaml_struct['Analysis'].get('scenarios', [])
scenarios.append('default')

runs = []

for scenario in scenarios:

    print (f'running scenario {scenario}')
    model_output_directory = model_run_base_directory + f"/{scenario}"
    if not os.path.exists(model_output_directory):
        os.makedirs(model_output_directory)

    create_model_func, sim_control, yaml_struct = prepare_simulation(model_output_directory,
                                                                 simulation_run_description, yaml_struct,
                                                                 scenario)

    runner = SimulationRunner()
    model, footprint_result_dict = runner.run(create_model_func=create_model_func,
                                              sim_control=sim_control,
                                            debug=True,
                                              target_units=YamlLoader.get_target_units(yaml_struct),
                                             result_variables=yaml_struct['Analysis'].get('result_variables', []))
    
    runs.append((scenario, model, footprint_result_dict, sim_control, yaml_struct));


[root                ][INFO   ]  Running with parameters <__main__.argsview object at 0x7f8148b96f70> (3327315392.py:10)
[root                ][INFO   ]  Running with local parameter data only. (yaml_runner.py:403)


KeyError: 'Analysis'

In [ ]:
i_vars = model.collect_input_variables()

In [ ]:
type(i_vars['Playout']['mean_power_per_linear_channel'])

In [ ]:
output_directory = sim_control.output_directory
base_dir = '.'
metadata = load_metadata(output_directory, base_dir=base_dir)

xlsx_file_name = f'{base_dir}/{output_directory}/results_{scenario}_{time.strftime("%m%d-%H%M")}.xlsx'
writer = pd.ExcelWriter(xlsx_file_name)
sheet_descriptions = {}
pd.DataFrame.from_dict(sheet_descriptions, orient='index').to_excel(writer, 'toc')
# df of x samples, monthly frequency between start and end date
load_data = load_as_df_qantity
start_date = yaml_struct['Metadata']['start_date']
end_date = yaml_struct['Metadata']['end_date']

common_args = {'start_date': start_date,
                       'end_date': end_date,
                       'base_dir': base_dir,
                       'metadata': metadata,
                       'output_scenario_directory': output_directory}

In [ ]:
variable = 'energy'

In [ ]:
unit = str(q_data.units)

In [ ]:
q_data = load_data(f'{output_directory}/result_data_{variable}.hdf5')
data = q_data.m



In [ ]:

name = plot_def['name']
variable = plot_def['variable']
logger.info(plot_def['name'])

# sum up monthly values to aggregate - duration depends on distance between start and end date
#    load_data_aggegrate = lambda : load_data().sum(level='samples')

q_data = load_data(f'{output_directory}/result_data_{variable}.hdf5')
data = q_data.m
unit = str(data.units)
xlabel = f'{unit}/a'
common_args['xlabel']= xlabel        

if 'groups' in plot_def:
    data = group_data(data, metadata, plot_def)



In [ ]:
ax.set_ylim([0,20000])

In [ ]:
data.columns[0]

In [ ]:

variable = 'energy'
q_data = load_data(f'{output_directory}/result_data_{variable}.hdf5')
data = q_data.m
# get number of processes
len(data.columns) 

In [ ]:
import math
rows = math.ceil(len(data.columns) / 3)

In [ ]:
yaml_struct['Metadata']

In [ ]:
data.abs().groupby(level=['time']).quantile(.75).values.max()

In [ ]:
data.abs().groupby(level=['time']).quantile(.75).index

In [ ]:
data.head(2)

In [ ]:
data.abs().groupby(level=['time']).quantile(.75).head(2)

In [ ]:
data = data.ix[:, data.abs().groupby(level=['time']).quantile(.75).max().sort_values(ascending=False).index]


In [ ]:
# joint max of 75th percentile 
# yhigh = data.abs().groupby(level=['time']).quantile(.75).values.max() * 1.1

# joint min of 25th percentile 
# ylow = data.abs().groupby(level=['time']).quantile(.75).values.max() * 1.1

import matplotlib
import matplotlib.cm as cm

maxima = data[data.columns[0]].abs().groupby(level=['time']).quantile(.75).mean()
minima = data[data.columns[-1]].abs().groupby(level=['time']).quantile(.75).mean()

norm = matplotlib.colors.LogNorm(vmin=minima, vmax=maxima, clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.viridis_r)


In [ ]:
plt.close('all')

# sort
data = data.ix[:, data.abs().groupby(level=['time']).quantile(.75).max().sort_values(ascending=False).index]

n = len(data.columns)
# fig = plt.figure(figsize=(12,n*0.75))
rows = math.ceil(n/ 3)
f, axarr = plt.subplots(rows,3,sharex='col', sharey='row')
f.set_size_inches(10,n*0.5)

# joint max of 75th percentile 
# yhigh = data.abs().groupby(level=['time']).quantile(.75).values.max() * 1.1

# joint min of 25th percentile 
# ylow = data.abs().groupby(level=['time']).quantile(.75).values.max() * 1.1



for i, ax in enumerate(axarr.flat):    
    
    ax.set_title(data.columns[i])
    process_name = data.columns[i]
#     print(f'{i}: {process_name}')
    
    process_data = data[process_name]
    mean_ = process_data.abs().mean(level='time')
    
    grouped_ = process_data.abs().groupby(level=['time'])
    
    low = grouped_.quantile(.25)  
    high = grouped_.quantile(.75)
    
    mean_.plot(ax=ax,
                                   kind='area',
                                   legend=False,
                                   linewidth=0.5, 
                                   color=mapper.to_rgba(high.mean())
              )
    
    low.plot(ax=ax, color='k',alpha=0.2, linestyle=':')
    high.plot(ax=ax, color='k',alpha=0.2, linestyle=':') 
    ax.fill_between(mean_.index.get_level_values(0), 
                     low, 
                     high, 
                     facecolor='k', 
#                      hatch="+", 
                     edgecolor="k", linewidth=0.1,
                     alpha=0.1, interpolate=True, linestyle='-')
#     ylim= process_data.abs().groupby(level=['time']).quantile(.75).values.max()
#     print(f'{ylim}')
    ax.set_xlabel('')
f.text(0.5, -0.0, 'time', ha='center')
f.text(-0.0, 0.5, unit, va='center', rotation='vertical')

plt.tight_layout()
    # data.abs().sum(axis=1).groupby(level=['time']).quantile(.25).plot(ax=ax, color='red')

In [ ]:
f.text(0.5, -0.0, 'time', ha='center')
f.text(-0.0, 0.5, 'MWh', va='center', rotation='vertical')

In [ ]:
len(axarr.flat)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Simple data to display in various forms
x = np.linspace(0, 2 * np.pi, 400)
y = np.sin(x ** 2)

plt.close('all')

In [ ]:
plt.close('all')

In [ ]:
n = 4
fig = plt.figure(figsize=(12,n*0.75))
rows = math.ceil(n/ 3)
f, axarr = plt.subplots(rows, 3,sharex='col', sharey='row')

for i, ax in enumerate(axarr.flat):    
    for ax in axarr.flat:
        ax.set_title('test')
        ax.plot(x, y)
#         ax.set(xlabel='x-label', ylabel='y-label')
plt.tight_layout()

#     axarr[0, 0].plot(x, y)
# # f, axarr = plt.subplots(2, 2)
# axarr[0, 0].plot(x, y)
# axarr[0, 0].set_title('Axis [0,0]')
# axarr[0, 1].scatter(x, y)
# axarr[0, 1].set_title('Axis [0,1]')
# axarr[1, 0].plot(x, y ** 2)
# axarr[1, 0].set_title('Axis [1,0]')
# axarr[1, 1].scatter(x, y ** 2)
# axarr[1, 1].set_title('Axis [1,1]')
# for ax in axarr.flat:
#     ax.set(xlabel='x-label', ylabel='y-label')


In [ ]:
fig.set_size_inches(12a.5, 20.5)

In [ ]:
data.abs().mean(level='time').plot(ax=ax,
                               kind='area',
                               # legend=False,
                               linewidth=0.5, colormap='Paired')

In [ ]:
low = data.abs().sum(axis=1).groupby(level=['time']).quantile(.25)
low.plot(ax=ax, color='k',alpha=0.2, linestyle=':')

In [ ]:
high = data.abs().sum(axis=1).groupby(level=['time']).quantile(.75)
high.plot(ax=ax, color='k',alpha=0.2, linestyle=':')


In [ ]:
ylim = high.max()
ax.set_ylim([0,ylim*1.1])

In [ ]:
plt.fill_between(data.mean(level='time').index.get_level_values(0), 
                 low, 
                 high, 
                 facecolor='k', 
                 hatch="+", 
                 edgecolor="k", linewidth=0.1,
                 alpha=0.1, interpolate=True, linestyle='-')
# data.abs().sum(axis=1).groupby(level=['time']).quantile(.25).plot(ax=ax, color='red')


In [ ]:
ax.legend(loc='center left', bbox_to_anchor=(.8, 0.5),
          ncol=1, fancybox=True, shadow=True)

In [ ]:
data.mean(level='time').index.get_level_values(0)

In [ ]:
data.plot.area()

In [ ]:
data.to_pickle(f'{output_directory}/graph_data_{name}.pdpkl')
d = plot_kind(data, figsize=(15, 12), file_name=f'{scenario}_{name}.pdf', title='Annual Total',
                          kind=plot_def.get('kind', 'box'), **common_args)

In [ ]:
data = load_data(f'{output_directory}/result_data_{variable}.hdf5')

In [ ]:
data['Google Origin DC'].index

In [ ]:
data

In [ ]:
# data.index = data.index.droplevel(1)
data.plot.area()


In [ ]:
import copy
rcParams_bkp = copy.deepcopy(plt.rcParams)
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.linewidth'] = .2
plt.rcParams['axes.spines.top'] = True
plt.rcParams['axes.spines.right'] = True
plt.rcParams['axes.spines.bottom'] = True
plt.rcParams['axes.spines.left'] = True
plt.rcParams['font.size'] = 13
plt.rcParams['grid.linestyle'] = ':'
plt.rcParams['lines.linewidth'] = .1
plt.rcParams['hatch.linewidth'] = 0.3

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)

title = f'{title} ({kwargs["start_date"]} - {kwargs["end_date"]})'

ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)
df.reindex_axis(df.mean().sort_values().index, axis=1)
df.T.plot(ax=ax, kind='barh', legend=False, color='w', edgecolor='k', align='center', width=0.5,
          linewidth=0.5, title=title)
bars = ax.patches
patterns = ['///', '--', '...', '\///', 'xxx', '\\\\']
hatches = [p for p in patterns for i in range(len(df))]
for bar, hatch in zip(bars, hatches):
    bar.set_hatch(hatch)

from textwrap import wrap
ax.set_yticklabels(['\n'.join(wrap(l.get_text(), 10)) for l in ax.get_yticklabels()])

if file_name:
    if not os.path.exists(f'{base_dir}/{output_scenario_directory}'):
        os.makedirs(f'{base_dir}/{output_scenario_directory}')
    file_name_ = f'{base_dir}/{output_scenario_directory}/{file_name}'
    logger.info(f'storing plot at {file_name_}')
    fig.savefig(file_name_)
# restore
plt.rcParams.update(rcParams_bkp)

In [ ]:
from ngmodel.common_graphical_analysis import plot_box, load_metadata
from ngmodel.util import load_as_df_qantity
from scenarios.graphical_analysis import sum_interval, group_by


In [ ]:
scenario=model.name
output_directory = sim_control.output_directory
base_dir='.'

if 'start_date' in yaml_struct['Metadata']:
    start_date = yaml_struct['Metadata']['start_date']
    end_date = yaml_struct['Metadata']['end_date']
    
metadata = load_metadata(output_directory, base_dir=base_dir)

load_data = partial(load_as_df_qantity, f'{output_directory}/result_data.hdf5')

# Plot all processes

Analysis:
   plots:
    - name: device_groups
      groups:
        - name: Wired Access Network
          included_types:
          - Wired Access Network
        - name: User Devices
          included_types:
          - Viewing Device

In [ ]:
plot_defs = yaml_struct['Analysis']['plots']

for plot_def in plot_defs:
    name = plot_def['name']
    logger.info(plot_def['name'])
    
    data = load_data()
    unit = str(data.units)

    # sum up monthly values to aggregate - duration depends on distance between start and end date
    #    load_data_aggegrate = lambda : load_data().sum(level='samples')
    xlabel = f'{unit}/a'
    common_args = {'start_date': start_date,
                   'end_date': end_date,
                   'base_dir': base_dir,
                   'xlabel': xlabel,
                   'metadata': metadata,
                   'output_scenario_directory': output_directory}
    data = sum_interval(data, start_date, end_date)
    
    if 'groups' in plot_def:
        for group in plot_def['groups']:
            group_name  = group['name']
            
            kwargs = {}
            if 'included_types' in group:
                included_types = group['included_types']
                kwargs.update({'included_types':included_types})
            
            data = group_by(data, metadata=metadata, group_name=group_name, **kwargs)
    
    d = plot_box(data, figsize=(15, 12), file_name=f'{scenario}_{name}.pdf', title='annual total',
             **common_args)
    



In [ ]:
# df = group_user_devices(data, metadata=metadata)
df = sum_interval(data, start_date, end_date)
d = plot_box(df, figsize=(15, 12), file_name=f'{scenario}_plot_all_processes_annual.pdf', title='annual total',
             **common_args)

# Plot with Groups

In [ ]:
plot_defs = yaml_struct['Analysis']['plots']
plot_defs

In [ ]:
data = pd.DataFrame(data=[[1,2,3]], columns=['a','b','c'], index=[1,2,3])
data

In [ ]:
metadata = {'a':{'device_type': 'Wired Access Network'},
            'b':{'platform_name': 'DSL'},
            'c':{'device_type': 'Viewing Device'}}

In [ ]:
def group_by(df, pattern_list=None, group_name=None, metadata=None, included_types=None):    
    ud_cols = []
    for col in df.columns:
        if not col in metadata:
            # a column might be the result of a previous call to group_by. We do not want to include them in future operations.
            # thus exclude them by checking that the name is present in the simulation metadata
            continue
        md = metadata[col]

        # check the device type is correct
        if not included_types or 'device_type' in md and md['device_type'] in included_types:
            # check the name pattern matches
            if not pattern_list or any(ud_name.lower() in col.lower() for ud_name in pattern_list):
                ud_cols.append(col)

    df[group_name] = df[ud_cols].sum(axis=1)
    ud_df = df.drop(ud_cols, axis=1)
    return ud_df
df = group_by(data, metadata=metadata, group_name='TVs', included_types=['Viewing Device'], pattern_list=['c'])
df

In [ ]:
all_vars = model.collect_input_variables()

In [ ]:
ndfs = {}
for proc_name, vars in sorted(all_vars.items()):
    p_df = None
    for var_name, var in sorted(vars.items()):
        if 'energy' in var_name:
            continue
#         print(proc_name, var_name, var)
        if p_df is None:
            p_df = pd.DataFrame(data=var.m)            
            p_df.columns = [var_name]
        else:
            p_df[var_name] = var.m
    
    if not isinstance(p_df.index, pd.MultiIndex):
        p_df.index = sim_control._df_multi_index
    normalized_df=(p_df-p_df.min())/(p_df.max()-p_df.min())
    
    ndfs[proc_name] = normalized_df


In [ ]:
list(ndfs.keys())

In [ ]:
process_name = '4_5G Modem Router'

In [ ]:
mean_.head(2)

In [ ]:
ndfs[process_name].mean(level='time').plot.line()

In [ ]:
ndfs[process_name].columns

In [ ]:
process_data = data[process_name]
mean_ = process_data.abs().mean(level='time')

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)

grouped_ = process_data.abs().groupby(level=['time'])

low = grouped_.quantile(.25)  
high = grouped_.quantile(.75)

mean_.plot(ax=ax, kind='area',
                               legend=False,
                               linewidth=0.5, 
                               color='k', alpha=.3
           
          )

ndfs[process_name].mean(level='time').plot.line(ax=ax, secondary_y=True, linewidth=0.3,)

low.plot(ax=ax, color='k',alpha=0.2, linestyle=':')
high.plot(ax=ax, color='k',alpha=0.2, linestyle=':') 
ax.fill_between(mean_.index.get_level_values(0), 
                 low, 
                 high, 
                 facecolor='k', 
#                      hatch="+", 
                 edgecolor="k", linewidth=0.1,
                 alpha=0.1, interpolate=True, linestyle='-')


ylim= process_data.abs().groupby(level=['time']).quantile(.75).values.max()
#     print(f'{ylim}')
# ax.set_xlabel('')

# compare scenarios

In [ ]:
%%capture
args = {'yamlfile':'models/scenarios.yml', 'comment':'jupyter','local':True,       'filetype':'pdf','docker':True }

class argsview(object):
    def __init__(self, d):
        self.__dict__ = d
        
args = argsview(args)   

In [ ]:
model_run_base_directory, simulation_run_description, yaml_struct = load_configuration(args)

scenarios = yaml_struct['Analysis'].get('scenarios', [])
scenarios.append('default')

model_name = None

runners = {}

for scenario in scenarios:
    model_output_directory = model_run_base_directory + f"/{scenario}"
    if not os.path.exists(model_output_directory):
        os.makedirs(model_output_directory)

    create_model_func, sim_control, yaml_struct = prepare_simulation(model_output_directory,
                                                                     simulation_run_description, yaml_struct,
                                                                     scenario)

    runner = SimulationRunner()
    runner.run(create_model_func=create_model_func,
               sim_control=sim_control, debug=True,
               target_units=YamlLoader.get_target_units(yaml_struct),
               result_variables=yaml_struct['Analysis'].get('result_variables', []))

    # analysis(runner.model, runner.sim_control, yaml_struct)
    model_name = runner.model.name
    runners[scenario] = runner

In [ ]:
for scenario_name, run_data in runners.items():
    print(scenario_name)

In [ ]:
run = runners['default']

In [ ]:
scenario_name = 'AO_10'

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)


output_directory = run_data.sim_control.output_directory
q_data_a = load_data(f'{output_directory}/result_data_{variable}.hdf5')
unit = str(q_data_a.units)

data = q_data_a.m
base_dir = '.'
metadata = load_metadata(output_directory, base_dir=base_dir)

# if 'groups' in plot_def:
#     data = group_data(data, metadata, plot_def)

mean_ = data.abs().mean(level='time').sum(axis=1)

mean_.plot(ax=ax, kind='area',
#            legend=False,
           linewidth=0.5,
           color='k', alpha=.3
           )



In [ ]:
# load data a,
# load data b
load_data = load_as_df_qantity
variable = 'energy'
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)

for scenario_name, run_data in runners.items():
    output_directory = run_data.sim_control.output_directory
    q_data_a = load_data(f'{output_directory}/result_data_{variable}.hdf5')
    unit = str(q_data_a.units)

    data = q_data_a.m
    base_dir = '.'
    metadata = load_metadata(output_directory, base_dir=base_dir)

    # if 'groups' in plot_def:
    #     data = group_data(data, metadata, plot_def)

    mean_ = data.abs().mean(level='time').sum(axis=1)

    mean_.plot(ax=ax, kind='line',
#                legend=False,
               linewidth=1,
               alpha=.3,
               label=scenario_name
               )
    
ax.legend()
    
plt.tight_layout()
file_name = 'scenarios.pdf'
if file_name:
    if not os.path.exists(f'{base_dir}/{model_run_base_directory}'):
        os.makedirs(f'{base_dir}/{model_run_base_directory}')
    file_name_ = f'{base_dir}/{model_run_base_directory}/{file_name}'
    logger.info(f'storing plot at {file_name_}')
    fig.savefig(file_name_)

In [ ]:
all_vars = run.model.collect_input_variables()

In [ ]:
for n in model.process_graph.nodes():
    print(n.name)

In [ ]:
sorted(list(all_vars.keys()))

In [ ]:
data

In [ ]:
_vars = {}

for _, p_vars in all_vars.items():
    _vars.update(p_vars)

In [ ]:

p_df = pd.DataFrame(data=_vars['total_iplayer_tablet_device_bits_per_day'].m)
p_df.columns = ['total_iplayer_tablet_device_bits_per_day']

In [ ]:
mean_ = p_df.mean(level='time')
mean_.plot(kind='line',
                   legend=False,
                   linewidth=1,
                   color='k', alpha=.3,
                   marker="x"
                   )

In [ ]:
_vars.keys()


In [ ]:
_vars['adsl_access_network_energy_per_hh_per_reference_period']

In [ ]:
sorted(_vars.keys())[0:11]

In [ ]:
import math
n = len(_vars)
# n = 10
rows = math.ceil(n / 3)
f, axarr = plt.subplots(rows, 3, sharex='col', sharey=False)
f.set_size_inches(10, n * 0.5)

var_names = sorted(_vars.keys())
for i, ax in enumerate(axarr.flat):
    var_name = var_names[i]
    val = _vars[var_name]
# for var_name, val in sorted(_vars.items()):
#     fig = plt.figure(figsize=(10, 5))
#     ax = fig.add_subplot(111)

    data = _vars[var_name].to_reduced_units()

    p_df = pd.DataFrame(data=data.m)
    p_df.columns = [var_name]
    if not isinstance(p_df.index, pd.MultiIndex):
        p_df.index = run.sim_control._df_multi_index
        
    mean_ = p_df.mean(level='time')
    mean_.plot(ax=ax, kind='line',
                   legend=False,
                   linewidth=1,
                   color='k', alpha=.3,
#                    marker="x"
                   )
    fig.suptitle(var_name, fontsize=16)

    ax.set_ylabel(data.units)
#     fig.title(var_name)
    ax.set_title(var_name [:15])
    plt.subplots_adjust(hspace=0.4, wspace=.3)
    fig.tight_layout()
    